## Notebook Magic

In [2]:
%matplotlib inline
%load_ext autoreload

## Imports

In [3]:
import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

## Get Intervencoes Table

In [3]:
interv = pd.read_parquet("s3://iefp-unemployment/intermediate/clean/interventions_small.parquet")

In [4]:
interv.shape # 10 mill

(10000000, 26)

## Extract Recommendable Interventions

In [5]:
intervention_codes = yaml.load(open("../conf/base/recommendable_interventions.yml"), Loader=yaml.FullLoader)

In [6]:
interv = interv[interv["codigo_intervencao"].isin(intervention_codes)]

In [7]:
interv.shape # 1,611,788

(1611788, 26)

## Get Journeys Table

In [8]:
journeys = pd.read_parquet("s3://iefp-unemployment/intermediate/transform/journeys_a.parquet")

In [9]:
journeys = journeys.set_index('user_id')

In [10]:
journeys.shape # 1,289,627

(1289627, 28)

In [11]:
journeys.loc[2158897]

,journey_count,register_date,exit_date_21,exit_date_31,exit_reason,register_reason,birth_date,gender,age,disabled,...,parish,center,subsidy,rsi,previous_cpp,previous_job_experience,desired_cpp,desired_location,desired_work_time,desired_contract
user_id,,,,,,,,,,,,,,,,,,,,,
2158897,1.0,2018-02-02,2018-05-21,NaT,NaN,8.0,1980-12-13,M,37.0,0.0,...,121502,474.0,54.0,True,71121.0,240.0,71121.0,C,C,P
2158897,2.0,2018-05-21,NaT,2018-05-21,20.0,8.0,1980-12-13,M,37.0,0.0,...,121502,474.0,24.0,True,71121.0,240.0,71121.0,C,C,P
2158897,3.0,2018-05-21,NaT,2018-05-30,20.0,8.0,1980-12-13,M,37.0,0.0,...,121502,474.0,24.0,True,71121.0,240.0,71121.0,C,C,P
2158897,4.0,2018-10-31,NaT,2019-01-29,20.0,8.0,NaT,None,NaN,NaN,...,None,NaN,NaN,None,NaN,NaN,NaN,None,None,None


## Sort interventions and extract tipo_movimento 35

In [12]:
interv = interv.sort_values(["ute_id", "data_intervencao"])
interv.shape

(1611788, 26)

In [13]:
interv = interv[interv.tipo_movimento == 35]
interv.shape

(393153, 26)

In [14]:
interv.codigo_intervencao = interv.codigo_intervencao.astype('str')

In [15]:
#make unique key in interventions table 
interv['unique_interv'] = range(1, len(interv.index)+1)

## Join to journeys table using date

In [16]:
interv = interv[["ute_id", "data_intervencao", "codigo_intervencao", "resultado_intervencao", "unique_interv"]]

In [17]:
interv = interv.set_index('ute_id')

In [18]:
interv.head()

,data_intervencao,codigo_intervencao,resultado_intervencao,unique_interv
ute_id,,,,
11,2005-12-07,501,1.0,1
38,2016-10-17,772,2.0,2
38,2018-03-19,225,39.0,3
56,2018-06-03,394,90.0,4
108,2017-02-21,507,1.0,5


In [41]:
journeys.head()

register_date exit_date_21 exit_date_31  exit_reason  \
user_id journey_count                                                        
38      1.0              2017-10-04          NaT   2018-01-15         20.0   
        2.0              2018-02-09   2018-06-08          NaT          NaN   
44      1.0              2016-12-06   2017-06-08          NaT          NaN   
56      1.0              2016-10-31          NaT   2017-01-20         20.0   
        2.0              2018-01-15          NaT   2019-01-03         61.0   

                       register_reason birth_date gender   age  disabled  \
user_id journey_count                                                      
38      1.0                        5.0 1973-06-11      M  44.0       0.0   
        2.0                        8.0 1973-06-11      M  44.0       0.0   
44      1.0                        8.0 1972-07-06      M  44.0       0.0   
56      1.0                        8.0 1979-01-25      M  37.0       0.0   
        2.0                        5.0 1979-01-25      M  38.0       0.0   

                       dependents  ...  parish center subsidy   rsi  \
user_id journey_count              ...                                
38      1.0                   1.0  ...  111012  336.0    26.0  True   
        2.0                   0.0  ...  111012  336.0    24.0  True   
44      1.0                   2.0  ...  110508  336.0    22.0  True   
56      1.0                   0.0  ...  130324  119.0     NaN  True   
        2.0                   4.0  ...  130324  119.0     NaN  True   

                      previous_cpp previous_job_experience desired_cpp  \
user_id journey_count                                                    
38      1.0                31140.0                    36.0     31140.0   
        2.0                31140.0                    36.0     31140.0   
44      1.0                52490.0                    36.0     52490.0   
56      1.0                83420.0                   120.0     83420.0   
        2.0                41100.0                    12.0     41100.0   

                      desired_location  desired_work_time  desired_contract  
user_id journey_count                                                        
38      1.0                       None                  C                 T  
        2.0                       None                  C                 T  
44      1.0                          C                  C                 P  
56      1.0                          C                  P                 T  
        2.0                          C                  C                 P  

[5 rows x 27 columns]

In [ ]:
# inner join
join_inner = journeys.iloc[:,0:5].merge(interv, how='inner', left_index=True, right_index=True)
join_inner.shape #261,242

In [20]:
join_inner.head()

,journey_count,register_date,exit_date_21,exit_date_31,exit_reason,data_intervencao,codigo_intervencao,resultado_intervencao,unique_interv
38,1.0,2017-10-04,NaT,2018-01-15,20.0,2016-10-17,772,2.0,2
38,1.0,2017-10-04,NaT,2018-01-15,20.0,2018-03-19,225,39.0,3
38,2.0,2018-02-09,2018-06-08,NaT,NaN,2016-10-17,772,2.0,2
38,2.0,2018-02-09,2018-06-08,NaT,NaN,2018-03-19,225,39.0,3
56,1.0,2016-10-31,NaT,2017-01-20,20.0,2018-06-03,394,90.0,4


## filter out interventions that don't occur within journey dates

In [21]:
join_inner = join_inner[(join_inner.data_intervencao >= join_inner.register_date) 
                        &((join_inner.data_intervencao <= join_inner.exit_date_21)
                        |(join_inner.data_intervencao <= join_inner.exit_date_31))]

In [22]:
join_inner[join_inner.unique_interv == 106111]

,journey_count,register_date,exit_date_21,exit_date_31,exit_reason,data_intervencao,codigo_intervencao,resultado_intervencao,unique_interv
2158897,1.0,2018-02-02,2018-05-21,NaT,NaN,2018-05-21,230,39.0,106111
2158897,2.0,2018-05-21,NaT,2018-05-21,20.0,2018-05-21,230,39.0,106111
2158897,3.0,2018-05-21,NaT,2018-05-30,20.0,2018-05-21,230,39.0,106111


In [23]:
join_inner[(join_inner.register_date == join_inner.exit_date_31) | (join_inner.register_date == join_inner.exit_date_21)].head(8)

,journey_count,register_date,exit_date_21,exit_date_31,exit_reason,data_intervencao,codigo_intervencao,resultado_intervencao,unique_interv
9185,1.0,2016-11-03,2017-11-26,2016-11-03,99.0,2017-01-03,506,25.0,342
66648,1.0,2016-10-23,2016-10-23,2019-01-10,44.0,2018-06-18,225,39.0,2146
114002,5.0,2018-12-04,2019-02-12,2018-12-04,99.0,2019-01-25,7005,1.0,3605
141190,1.0,2016-10-02,2016-10-02,2018-12-12,11.0,2016-10-31,770,1.0,4815
173438,1.0,2018-02-21,NaT,2018-02-21,61.0,2018-02-21,327,24.0,5804
199535,1.0,2016-09-22,2016-09-22,2019-05-27,11.0,2018-04-10,230,86.0,6710
199535,1.0,2016-09-22,2016-09-22,2019-05-27,11.0,2018-04-10,225,39.0,6711
226552,1.0,2016-11-30,2018-07-30,2016-11-30,20.0,2017-11-24,506,1.0,7604


## Filter sucessful interventions

In [24]:
sucess = [1,3,4,5,8,9,10,11,12,14,15,20,31,54,55,56,88,91,98]
join_inner = join_inner[join_inner.resultado_intervencao.isin(sucess)]
join_inner.shape

(60901, 9)

In [25]:
join_inner.codigo_intervencao.drop_duplicates().shape

(60,)

## Pivot table

In [26]:
join_inner = join_inner.reset_index().rename(columns={"index": "user_id"})

In [27]:
join_inner.head()
join_inner = join_inner[["user_id","journey_count","data_intervencao","codigo_intervencao"]]

In [28]:
join_inner = join_inner = join_inner.set_index(["user_id","journey_count"])

In [29]:
join_inner = join_inner.pivot_table(index = ["user_id", "journey_count"], columns = ["codigo_intervencao"], aggfunc=np.min)


In [30]:
join_inner.columns = ['_'.join(col).strip() for col in join_inner.columns.values]

In [31]:
join_inner.head()

,,data_intervencao_207,data_intervencao_208,data_intervencao_218,data_intervencao_219,data_intervencao_225,data_intervencao_226,data_intervencao_235,data_intervencao_319,data_intervencao_321,data_intervencao_323,...,data_intervencao_791,data_intervencao_794,data_intervencao_795,data_intervencao_796,data_intervencao_797,data_intervencao_9902,data_intervencao_9903,data_intervencao_9904,data_intervencao_9906,data_intervencao_9907
user_id,journey_count,,,,,,,,,,,,,,,,,,,,,
574,1.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2018-05-18,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
724,3.0,NaT,2018-06-05,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
749,1.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1687,2.0,NaT,2018-08-09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1725,2.0,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [33]:
join_inner.shape

(55454, 60)

## Join to journeys table

In [37]:
journeys.head()

register_date exit_date_21 exit_date_31  exit_reason  \
user_id journey_count                                                        
38      1.0              2017-10-04          NaT   2018-01-15         20.0   
        2.0              2018-02-09   2018-06-08          NaT          NaN   
44      1.0              2016-12-06   2017-06-08          NaT          NaN   
56      1.0              2016-10-31          NaT   2017-01-20         20.0   
        2.0              2018-01-15          NaT   2019-01-03         61.0   

                       register_reason birth_date gender   age  disabled  \
user_id journey_count                                                      
38      1.0                        5.0 1973-06-11      M  44.0       0.0   
        2.0                        8.0 1973-06-11      M  44.0       0.0   
44      1.0                        8.0 1972-07-06      M  44.0       0.0   
56      1.0                        8.0 1979-01-25      M  37.0       0.0   
        2.0                        5.0 1979-01-25      M  38.0       0.0   

                       dependents  ...  parish center subsidy   rsi  \
user_id journey_count              ...                                
38      1.0                   1.0  ...  111012  336.0    26.0  True   
        2.0                   0.0  ...  111012  336.0    24.0  True   
44      1.0                   2.0  ...  110508  336.0    22.0  True   
56      1.0                   0.0  ...  130324  119.0     NaN  True   
        2.0                   4.0  ...  130324  119.0     NaN  True   

                      previous_cpp previous_job_experience desired_cpp  \
user_id journey_count                                                    
38      1.0                31140.0                    36.0     31140.0   
        2.0                31140.0                    36.0     31140.0   
44      1.0                52490.0                    36.0     52490.0   
56      1.0                83420.0                   120.0     83420.0   
        2.0                41100.0                    12.0     41100.0   

                      desired_location  desired_work_time  desired_contract  
user_id journey_count                                                        
38      1.0                       None                  C                 T  
        2.0                       None                  C                 T  
44      1.0                          C                  C                 P  
56      1.0                          C                  P                 T  
        2.0                          C                  C                 P  

[5 rows x 27 columns]

In [36]:
journeys = journeys.reset_index().rename(columns={"index": "user_id"})
journeys = journeys.set_index(["user_id","journey_count"])

In [39]:
output = journeys.merge(join_inner, left_index = True, right_index = True, how = "inner")
output.head()

,,register_date,exit_date_21,exit_date_31,exit_reason,register_reason,birth_date,gender,age,disabled,dependents,...,data_intervencao_791,data_intervencao_794,data_intervencao_795,data_intervencao_796,data_intervencao_797,data_intervencao_9902,data_intervencao_9903,data_intervencao_9904,data_intervencao_9906,data_intervencao_9907
user_id,journey_count,,,,,,,,,,,,,,,,,,,,,
574,1.0,2017-06-30,NaT,2018-05-18,61.0,5.0,1976-05-06,M,41.0,0.0,2.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
724,3.0,2018-04-18,NaT,2018-12-19,44.0,15.0,1978-12-13,F,39.0,0.0,1.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
749,1.0,2017-05-23,2017-10-15,NaT,NaN,12.0,1967-02-11,F,50.0,0.0,1.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1687,2.0,2018-07-10,NaT,2019-02-07,20.0,8.0,1976-03-11,M,42.0,0.0,2.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1725,2.0,2016-10-21,NaT,2018-06-22,20.0,8.0,1977-06-04,M,39.0,0.0,1.0,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [40]:
output.shape

(55454, 87)

In [68]:
read = pd.read_parquet("s3://iefp-unemployment/intermediate/transform/transform_interventions.parquet")

In [69]:
read.head()

register_date exit_date_21 exit_date_31  exit_reason  \
user_id journey_count                                                        
38      1.0              2017-10-04          NaT   2018-01-15         20.0   
        2.0              2018-02-09   2018-06-08          NaT          NaN   
44      1.0              2016-12-06   2017-06-08          NaT          NaN   
56      1.0              2016-10-31          NaT   2017-01-20         20.0   
        2.0              2018-01-15          NaT   2019-01-03         61.0   

                       register_reason data_intervencao_101  \
user_id journey_count                                         
38      1.0                        5.0           2017-10-04   
        2.0                        8.0           2018-02-09   
44      1.0                        8.0                  NaT   
56      1.0                        8.0                  NaT   
        2.0                        5.0           2018-01-15   

                      data_intervencao_102 data_intervencao_103  \
user_id journey_count                                             
38      1.0                     2018-01-15                  NaT   
        2.0                     2018-03-22                  NaT   
44      1.0                     2017-06-08           2017-04-06   
56      1.0                     2017-01-20                  NaT   
        2.0                     2018-06-26           2018-04-06   

                      data_intervencao_207 data_intervencao_208  ...  \
user_id journey_count                                            ...   
38      1.0                            NaT                  NaT  ...   
        2.0                            NaT                  NaT  ...   
44      1.0                            NaT                  NaT  ...   
56      1.0                            NaT                  NaT  ...   
        2.0                            NaT                  NaT  ...   

                      data_intervencao_794 data_intervencao_795  \
user_id journey_count                                             
38      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   
44      1.0                            NaT                  NaT   
56      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   

                      data_intervencao_796 data_intervencao_797  \
user_id journey_count                                             
38      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   
44      1.0                            NaT                  NaT   
56      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   

                      data_intervencao_9902 data_intervencao_9903  \
user_id journey_count                                               
38      1.0                             NaT                   NaT   
        2.0                             NaT                   NaT   
44      1.0                             NaT                   NaT   
56      1.0                             NaT                   NaT   
        2.0                             NaT                   NaT   

                      data_intervencao_9904 data_intervencao_9906  \
user_id journey_count                                               
38      1.0                             NaT                   NaT   
        2.0                             NaT                   NaT   
44      1.0                             NaT                   NaT   
56      1.0                             NaT                   NaT   
        2.0                             NaT                   NaT   

                      data_intervencao_9907 data_intervencao_9999  
user_id journey_count                                              
38      1.0   

In [44]:
read.columns.values


array(['register_date', 'exit_date_21', 'exit_date_31', 'exit_reason',
       'register_reason', 'data_intervencao_101', 'data_intervencao_102',
       'data_intervencao_103', 'data_intervencao_207',
       'data_intervencao_208', 'data_intervencao_210',
       'data_intervencao_212', 'data_intervencao_218',
       'data_intervencao_219', 'data_intervencao_222',
       'data_intervencao_225', 'data_intervencao_226',
       'data_intervencao_231', 'data_intervencao_232',
       'data_intervencao_233', 'data_intervencao_234',
       'data_intervencao_235', 'data_intervencao_236',
       'data_intervencao_319', 'data_intervencao_321',
       'data_intervencao_323', 'data_intervencao_327',
       'data_intervencao_328', 'data_intervencao_329',
       'data_intervencao_394', 'data_intervencao_395',
       'data_intervencao_396', 'data_intervencao_398',
       'data_intervencao_399', 'data_intervencao_404',
       'data_intervencao_416', 'data_intervencao_427',
       'data_intervencao_428',

In [4]:
sucess = [1,3,4,5,8,9,10,11,12,14,15,20,31,54,55,56,88,91,98]
df_interv = pd.read_parquet("s3://iefp-unemployment/intermediate/clean/interventions_small.parquet")
df_journ = pd.read_parquet("s3://iefp-unemployment/intermediate/transform/journeys_a.parquet")

In [5]:
# Filter interventions
df_interv = df_interv[df_interv.tipo_movimento == 35]
df_interv = df_interv[df_interv.resultado_intervencao.isin(sucess)]
df_interv = df_interv[["ute_id", "data_intervencao", "codigo_intervencao"]]
df_interv.codigo_intervencao = df_interv.codigo_intervencao.astype('str') 
df_interv.head()       

,ute_id,data_intervencao,codigo_intervencao
1,88081,2019-07-01,226
3,6905559,2019-07-02,103
5,6905559,2019-07-02,212
6,6905559,2019-07-02,103
7,6905559,2019-07-02,103


In [6]:
# Set index's and merge with basic jouney info
df_interv = df_interv.set_index('ute_id') 
df_journ = df_journ.set_index('user_id')
df_out = df_journ.iloc[:,0:5].merge(df_interv, how='inner', left_index=True, right_index=True)
df_out.head()

,journey_count,register_date,exit_date_21,exit_date_31,exit_reason,data_intervencao,codigo_intervencao
38,1.0,2017-10-04,NaT,2018-01-15,20.0,NaT,7013
38,1.0,2017-10-04,NaT,2018-01-15,20.0,2018-03-22,102
38,1.0,2017-10-04,NaT,2018-01-15,20.0,2018-03-02,7001
38,2.0,2018-02-09,2018-06-08,NaT,NaN,NaT,7013
38,2.0,2018-02-09,2018-06-08,NaT,NaN,2018-03-22,102


In [7]:
# Filter for interventions that occur between journey dates
df_out = df_out[(df_out.data_intervencao >= df_out.register_date) 
                        &((df_out.data_intervencao <= df_out.exit_date_21)
                        |(df_out.data_intervencao <= df_out.exit_date_31))]
df_out = df_out[["journey_count","data_intervencao","codigo_intervencao"]]
df_out.head()

,journey_count,data_intervencao,codigo_intervencao
38,2.0,2018-03-22,102
38,2.0,2018-03-02,7001
56,2.0,2018-06-26,398
56,2.0,2018-06-26,102
56,2.0,2018-04-27,7001


In [8]:
# Pivot table
df_out = df_out.reset_index().rename(columns={"index": "user_id"})
df_out = df_out = df_out.set_index(["user_id","journey_count"])
df_out = df_out.pivot_table(index = ["user_id", "journey_count"], columns = ["codigo_intervencao"], aggfunc=np.min)
df_out.columns = ['_'.join(col).strip() for col in df_out.columns.values]
df_out.head()

data_intervencao_101 data_intervencao_102  \
user_id journey_count                                             
38      2.0                            NaT           2018-03-22   
56      2.0                            NaT           2018-06-26   
59      1.0                            NaT           2018-03-26   
        2.0                     2018-11-13           2019-03-12   
90      2.0                     2019-04-17           2019-05-10   

                      data_intervencao_103 data_intervencao_207  \
user_id journey_count                                             
38      2.0                            NaT                  NaT   
56      2.0                     2018-04-06                  NaT   
59      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   
90      2.0                            NaT                  NaT   

                      data_intervencao_208 data_intervencao_210  \
user_id journey_count                                             
38      2.0                            NaT                  NaT   
56      2.0                            NaT                  NaT   
59      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   
90      2.0                            NaT                  NaT   

                      data_intervencao_212 data_intervencao_218  \
user_id journey_count                                             
38      2.0                            NaT                  NaT   
56      2.0                            NaT                  NaT   
59      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   
90      2.0                            NaT                  NaT   

                      data_intervencao_219 data_intervencao_222  ...  \
user_id journey_count                                            ...   
38      2.0                            NaT                  NaT  ...   
56      2.0                            NaT                  NaT  ...   
59      1.0                            NaT                  NaT  ...   
        2.0                            NaT                  NaT  ...   
90      2.0                            NaT                  NaT  ...   

                      data_intervencao_794 data_intervencao_795  \
user_id journey_count                                             
38      2.0                            NaT                  NaT   
56      2.0                            NaT                  NaT   
59      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   
90      2.0                            NaT                  NaT   

                      data_intervencao_796 data_intervencao_797  \
user_id journey_count                                             
38      2.0                            NaT                  NaT   
56      2.0                            NaT                  NaT   
59      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   
90      2.0                            NaT                  NaT   

                      data_intervencao_9902 data_intervencao_9903  \
user_id journey_count                                               
38      2.0                             NaT                   NaT   
56      2.0                             NaT                   NaT   
59      1.0                             NaT                   NaT   
        2.0                             NaT                   NaT   
90      2.0                             NaT                   NaT   

                      data_intervencao_9904 data_intervencao_9906  \
user_id journey_count                                               
38      2.0                             NaT                   NaT   
56    

In [9]:
# Merge pivot table with full journey table
df_journ = df_journ.reset_index().rename(columns={"index": "user_id"})
df_journ = df_journ.set_index(["user_id","journey_count"])
df_out = df_out = df_journ.merge(df_out, left_index = True, right_index = True, how = "inner")
df_out.head()

register_date exit_date_21 exit_date_31  exit_reason  \
user_id journey_count                                                        
38      2.0              2018-02-09   2018-06-08          NaT          NaN   
56      2.0              2018-01-15          NaT   2019-01-03         61.0   
59      1.0              2017-12-20          NaT   2018-03-26         20.0   
        2.0              2018-11-13          NaT   2019-03-12         11.0   
90      2.0              2019-04-17          NaT   2019-05-10         20.0   

                       register_reason birth_date gender   age  disabled  \
user_id journey_count                                                      
38      2.0                        8.0 1973-06-11      M  44.0       0.0   
56      2.0                        5.0 1979-01-25      M  38.0       0.0   
59      1.0                        8.0 1971-09-14      F  46.0       0.0   
        2.0                        5.0 1971-09-14      F  47.0       0.0   
90      2.0                        7.0 1971-07-20      F  47.0       0.0   

                       dependents  ... data_intervencao_794  \
user_id journey_count              ...                        
38      2.0                   0.0  ...                  NaT   
56      2.0                   4.0  ...                  NaT   
59      1.0                   2.0  ...                  NaT   
        2.0                   2.0  ...                  NaT   
90      2.0                   0.0  ...                  NaT   

                      data_intervencao_795 data_intervencao_796  \
user_id journey_count                                             
38      2.0                            NaT                  NaT   
56      2.0                            NaT                  NaT   
59      1.0                            NaT                  NaT   
        2.0                            NaT                  NaT   
90      2.0                            NaT                  NaT   

                      data_intervencao_797 data_intervencao_9902  \
user_id journey_count                                              
38      2.0                            NaT                   NaT   
56      2.0                            NaT                   NaT   
59      1.0                            NaT                   NaT   
        2.0                            NaT                   NaT   
90      2.0                            NaT                   NaT   

                      data_intervencao_9903 data_intervencao_9904  \
user_id journey_count                                               
38      2.0                             NaT                   NaT   
56      2.0                             NaT                   NaT   
59      1.0                             NaT                   NaT   
        2.0                             NaT                   NaT   
90      2.0                             NaT                   NaT   

                      data_intervencao_9906  data_intervencao_9907  \
user_id journey_count                                                
38      2.0                             NaT                    NaT   
56      2.0                             NaT                    NaT   
59      1.0                             NaT                    NaT   
        2.0                             NaT                    NaT   
90      2.0                             NaT                    NaT   

                       data_intervencao_9999  
user_id journey_count                         
38      2.0                              NaT  
56      2.0                              NaT  
59      1.0                              NaT  
        2.0                              NaT  
90      2.0                              NaT  

[5 rows x 114 columns]

In [64]:
df_out.columns.values

array(['register_date', 'exit_date_21', 'exit_date_31', 'exit_reason',
       'register_reason', 'birth_date', 'gender', 'age', 'disabled',
       'dependents', 'professional_training', 'profession_card',
       'school_qualification', 'college_qualification', 'civil_status',
       'registration_type', 'nationality', 'parish', 'center', 'subsidy',
       'rsi', 'previous_cpp', 'previous_job_experience', 'desired_cpp',
       'desired_location', 'desired_work_time', 'desired_contract',
       'data_intervencao_101', 'data_intervencao_102',
       'data_intervencao_103', 'data_intervencao_207',
       'data_intervencao_208', 'data_intervencao_210',
       'data_intervencao_212', 'data_intervencao_218',
       'data_intervencao_219', 'data_intervencao_222',
       'data_intervencao_225', 'data_intervencao_226',
       'data_intervencao_231', 'data_intervencao_232',
       'data_intervencao_234', 'data_intervencao_235',
       'data_intervencao_236', 'data_intervencao_319',
       'data_i